# Mapping Ocean Global Salinity with Basemap

# Inline output

The following code helps make all of the code samples in this notebook display their output properly. 

Note that you need to run this cell before running any other cell in the notebook. Otherwise your output will display in a separate window, or it won't display at all. If you try to run a cell and the output does not display in the notebook:
- Restart the IPython Notebook kernel.
- Run the following cell.
- Run the cell you were interested in again.

In [ ]:
# This just lets the output of the following code samples
#  display inline on this page, at an appropriate size.
from pylab import rcParams
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
rcParams['figure.figsize'] = (12,8)

Python’s [matplotlib](http://matplotlib.org/) package is an amazing resource, and the [Basemap toolkit](http://matplotlib.org/basemap/) extends matplotlib’s capabilities to mapping applications.

Also, we will need the **netCDF** libray (<a ref='http://www.unidata.ucar.edu/netcdf/'>Network Common Data Form</a>) is a set of software libraries and self-describing, machine-independent data formats that support the creation, access, and sharing of array-oriented scientific data. The project homepage is hosted by the **Unidata** program at the University Corporation for Atmospheric Research (**UCAR**).

We will use them to make a map of ocean global salinity. The first thing consists in `importing` these libraries into our environment. 

In [ ]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.tri as Tri
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, shiftgrid
import netCDF4
import datetime as dt
import pandas as pd
from StringIO import StringIO

# Making a simple map

Let's start out by making a simple map of the world. If you run the following code, you should get a nice map of the globe, with good clean coastlines:

In [ ]:
# make sure the value of resolution is a lowercase L,
#  for 'low', not a numeral 1
my_map = Basemap(projection='ortho', lat_0=50, lon_0=-100,
              resolution='l', area_thresh=1000.0)
 
my_map.drawcoastlines()
 
plt.show()

# Adding details

Let’s add some more detail to this map, starting with country borders. Add the following lines after `map.drawcoastlines()`:

In [ ]:
# make sure the value of resolution is a lowercase L,
#  for 'low', not a numeral 1
my_map = Basemap(projection='ortho', lat_0=50, lon_0=-100,
              resolution='l', area_thresh=1000.0)
 
my_map.drawcoastlines()
my_map.drawcountries()
my_map.fillcontinents(color='green')
my_map.drawmapboundary()
 
plt.show()

Now let’s draw latitude and longitude lines:

In [ ]:
# make sure the value of resolution is a lowercase L,
#  for 'low', not a numeral 1
my_map = Basemap(projection='ortho', lat_0=50, lon_0=-100,
              resolution='l', area_thresh=1000.0)
 
my_map.drawcoastlines()
my_map.drawcountries()
my_map.fillcontinents(color='green')
my_map.drawmapboundary()
 
my_map.drawmeridians(np.arange(0, 360, 30))
my_map.drawparallels(np.arange(-90, 90, 30))
 
plt.show()

The `np.arange()` arguments tell where your latitude and longitude lines should begin and end, and how far apart they should be spaced.

Let’s play with two of the map settings, and then we'll move on to plotting data on this globe. Let’s start by adjusting the perspective. Change the latitude and longitude parameters in the original Basemap definition to -30 and 140. When you run the program, you should see your map centered along Australia:

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
 
# make sure the value of resolution is a lowercase L,
#  for 'low', not a numeral 1
my_map = Basemap(projection='ortho', lat_0=-30, lon_0=140,
              resolution='l', area_thresh=1000.0)
 
my_map.drawcoastlines()
my_map.drawcountries()
my_map.fillcontinents(color='green')
my_map.drawmapboundary()
 
my_map.drawmeridians(np.arange(0, 360, 30))
my_map.drawparallels(np.arange(-90, 90, 30))
 
plt.show()

Now let’s change the kind of map we're producing. Change the projection type to ‘robin’. You should end up with a Robinson projection instead of a globe:

In [ ]:
# make sure the value of resolution is a lowercase L,
#  for 'low', not a numeral 1
# try the following projection:
#- mill
#- robin
my_map = Basemap(projection='mill', lat_0=-30, lon_0=140,
              resolution='l', area_thresh=1000.0)
 
my_map.drawcoastlines()
my_map.drawcountries()
my_map.fillcontinents(color='green')

my_map.drawmapboundary()
 
my_map.drawmeridians(np.arange(0, 360, 30))
my_map.drawparallels(np.arange(-90, 90, 30))
 
plt.show()

Changing the color scale to bluemarble shows the color scale firstly introduced by NASA to highlight global differences in topography.

In [ ]:
color_map = Basemap(projection='moll', lat_0=-30, lon_0=140,
              resolution='l', area_thresh=1000.0)
 
color_map.drawcoastlines()
color_map.drawcountries()

# Define the new color map
color_map.bluemarble()
color_map.drawmapboundary()
 
color_map.drawmeridians(np.arange(0, 360, 30))
color_map.drawparallels(np.arange(-90, 90, 30))
 
plt.show()

In [ ]:
color_map = Basemap(projection='robin', lat_0=-30, lon_0=140,
              resolution='l', area_thresh=1000.0)
 
color_map.drawcoastlines()
color_map.drawcountries()

# Define the new color map
color_map.bluemarble()
color_map.drawmapboundary()
 
color_map.drawmeridians(np.arange(0, 360, 30))
color_map.drawparallels(np.arange(-90, 90, 30))
 
plt.show()

# Plotting global ocean salinity

As for our previous notebooks, we will use OPenDAP protocal to access the dataset.

#### OPenDAP: Open-source Project for a Network Data Access Protocol

In [ ]:
# Set the URL
url='http://thredds.jpl.nasa.gov/thredds/dodsC/ncml_aggregation/SalinityDensity/aquarius/aggregate__AQUARIUS_L3_SSS_SMI_7DAY_V4.ncml'
# Load it via OPeNDAP
nc = netCDF4.Dataset(url)
# Query the variables
nc.variables.keys()

Now we take the variables that we meed:
- longitude `lon`
- latitude `lat`
- salinity `l3m_data`

In [ ]:
slat = nc.variables['lat'][:]
slon = nc.variables['lon'][:]
sal = nc.variables['l3m_data'][0,:,:]

We then create a grid based on longitude and latitude coordinates.

In [ ]:
slon, slat = np.meshgrid(slon, slat)

We can now use Basemap to project our salinity data to a mew map.

In [ ]:
# initial map
backmap = Basemap(projection='robin',lon_0=0,resolution='c')

# salinity map
sal_map = Basemap(projection='robin',lon_0=0,resolution='c')
xsal, ysal = sal_map(slon,slat)

Time to draw the map

In [ ]:
sal_map.drawcoastlines(linewidth=0.25)
sal_map.drawcountries(linewidth=0.25)

sal_map.fillcontinents(color='#eeefff',lake_color='white')
sal_map.drawmapboundary(fill_color='white')
backmap.drawmeridians(np.arange(0,360,30))
backmap.drawparallels(np.arange(-90,90,30))

cs = backmap.contourf(xsal,ysal,sal, levels=np.arange(32,37.7,0.1), cmap=cm.spring,  vmin=32, vmax=37.5 )
cbar = sal_map.colorbar(cs,location='bottom',pad="5%")
cbar.set_label('salinity in psu')

# Closer look to our data for Australia

## Zooming in

Let’s see how to zoom in on a region. This is good to know because there are many data sets specific to one region of the world, which would get lost when plotted on a map of the whole world. Some projections can not be zoomed in at all, so if things are not working well, make sure to look at the [documentation](http://matplotlib.org/basemap/api/basemap_api.html).

Let zoom to Australia. One way to zoom in is to specify the latitude and longitude of the lower left and upper right corners of the region you want to show. Let’s use a mercator projection, which supports this method of zooming.

In [ ]:
aus_map = Basemap(projection='merc', lat_0=-33.8650, lon_0=151.2,
    resolution = 'l', area_thresh = 1000.0,
    llcrnrlon=110.0, llcrnrlat=-45,
    urcrnrlon=160.0, urcrnrlat=-10)
 
aus_map.drawcoastlines()
aus_map.drawcountries()
# Here we use the shaded relied color map
#aus_map.shadedrelief()
aus_map.drawmapboundary()
 
aus_map.drawmeridians(np.arange(0, 360, 30))
aus_map.drawparallels(np.arange(-90, 90, 30))
# Here we use the shaded relied color map
aus_map.etopo()

plt.show()

## Salinity map around Australia

First we `clip` our salinity dataset to the region of interest...

In [ ]:
# Write your code here

Now we plot the salinity data

In [ ]:
# Write your code here